## 练习对 Spark RDD 的操作

大家联系的时候也可以用 pyspark shell

### 初始化 pyspark 环境

In [2]:
#导包 这三行是固定的格式写法
from pyspark import SparkConf, SparkContext  

In [3]:
# 创建 spark 上下文（context）， 并配置
conf = SparkConf().setMaster("local").setAppName("practice_RDD")  # 当前阶段 setMaster 就填 local 本地模式， appName 随便写
sc = SparkContext(conf = conf)  # sc 这个变量是使用 Spark 的入口， 如果使用 shell， shell 会自动帮你把这两部做了
                                # scs 不能更改，惯用写法
                                #每次直接抄过来就可以了，只更改.setAppName("practice_RDD") 就可以了

### 常用的创建 RDD 的几种方式
1. 读文件
2. 从 python 的 list 转化
3. 直接创建

In [ ]:
RDD（Resilient Distributed Dataset）叫做弹性分布式数据集，是Spark中最基本的数据抽象，
它代表一个不可变、可分区、里面的元素可并行计算的集合。RDD具有数据流模型的特点：自动容错、位置感知性调度和
可伸缩性。RDD允许用户在执行多个查询时显式地将工作集缓存在内存中，后续的查询能够重用工作集，这极大地提升了


RDD是Spark数据结构最基本的抽象化概念之一。RDD是一个不可改写的数据体，immutable，你只可以从一个RDD经过变换
生产出一个新的RDD，你不能直接修改RDD里的数据。如果你熟悉functional programming应该可以理解这个概念。
一个RDD有3个必须的组成部分
1，每个RDD都有很多块构成，这个抽象化概念是为了允许RDD被分布式处理。用白话解释一下就是你有个1000行数据的RDD，
那这个RDD会由100块10行的部分组成，也可以由一块1000行的部分组成。关键是这些小块数据是RDD的组成部分。
2，我开头说的，你只可以从一个RDD变换生产出另一个RDD，所以每个RDD都会记录他是从哪个RDD生出来的。
RDD之间就产生了一个树状的关系。这个抽象化概念是为了数据恢复。假设你丢失了一块RDD数据，由于有了这个概念，
你可以找到丢掉的数据他妈，重新计算下你丢掉的那块数据。
3，第二点是说每个RDD都有个妈妈，第三点就是RDD是怎么生出来的。老王对RDD妈妈做了什么才造出了RDD儿子。
有这个概念也是为了数据恢复。打个比方，RDD_son是老妈RDD_mom做了一个map(x => x*2)把老妈的每个成员翻一倍产生的。
那这个.map(x => x*2)就是每个RDD的第三个组成部分。 
所以经过这三点你可以看出RDD这个抽象化概念完全是为了网络分布式运算产生了。首先他可以分成很多块，
找很多机器一人计算一块。其次，假设有台机器被偷了，有些数据丢了，你可以根据RDD的从属关系重新计算丢掉的数据。
RDD本身是一个interface，你只要能做到以上三点就可以写自己的RDD拿到Spark上运行。

In [1]:
Spark 中最基本的数据抽象是 RDD。

RDD：弹性分布式数据集 (Resilient Distributed DataSet)。

1，RDD 有三个基本特性
这三个特性分别为：分区，不可变，并行操作。
a， 分区
每一个 RDD 包含的数据被存储在系统的不同节点上。逻辑上我们可以将 RDD 理解成一个大的数组，数组中的每个元素就
代表一个分区 (Partition) 。在物理存储中，每个分区指向一个存储在内存或者硬盘中的数据块 (Block) ，
其实这个数据块就是每个 task 计算出的数据块，它们可以分布在不同的节点上。所以，RDD 只是抽象意义的数据集合，
分区内部并不会存储具体的数据，只会存储它在该 RDD 中的 index，通过该 RDD 的 ID 和分区的 index 可以唯一确定
对应数据块的编号，然后通过底层存储层的接口提取到数据进行处理。在集群中，各个节点上的数据块会尽可能的存储在
内存中，只有当内存没有空间时才会放入硬盘存储，这样可以最大化的减少硬盘 IO 的开销。

b，不可变
不可变性是指每个 RDD 都是只读的，它所包含的分区信息是不可变的。由于已有的 RDD 是不可变的，所以我们只有对
现有的 RDD 进行转化 (Transformation) 操作，才能得到新的 RDD ，一步一步的计算出我们想要的结果。
这样会带来这样的好处：我们在 RDD 的计算过程中，不需要立刻去存储计算出的数据本身，我们只要记录每个 RDD 
是经过哪些转化操作得来的，即：依赖关系，这样一方面可以提高计算效率，一方面是错误恢复会更加容易。如果在计算
过程中，第 N 步输出的 RDD 的节点发生故障，数据丢失，那么可以根据依赖关系从第 N-1 步去重新计算出该 RDD，
这也是 RDD 叫做**"弹性"**分布式数据集的一个原因。
c，并行操作
因为 RDD 的分区特性，所以其天然支持并行处理的特性。即不同节点上的数据可以分别被处理，然后生成一个新的 RDD。


SyntaxError: invalid syntax (<ipython-input-1-6f82df36f7db>, line 1)

In [6]:
#第一种
# 读文件，按行读取
# readFile = sc.textFile('')

In [3]:
#第二种
# 从 python 的 list 转化
# RDD：弹性分布式数据集 (Resilient Distributed DataSet)
l = [1, 3, 4, 6, "asd"]  #python的list
#然后使用parallelize() 这个方法，把python list 转化成rdd
fromPythonList = sc.parallelize(l)
#怎么把rdd 打印出来  collect() 必须这么做
fromPythonList.collect()

[1, 3, 4, 6, 'asd']

In [4]:
#然后使用parallelize() 这个方法，把python list 转化成rdd
#在转化的同时还可以直接进行分区操作
l = [1, 3, 4, 6, "asd"]  #python的list
#分成3个区
fromPythonList = sc.parallelize(l,3)
#glom() 查看分区
fromPythonList.glom().collect()
#查看分区数量
fromPythonList.getNumPartitions()

3

In [8]:
# 从 python 的 list 转化
l = [1, 3, 4, 6, "asd"]
sc.parallelize(l).collect()

[1, 3, 4, 6, 'asd']

In [9]:
#取前3个
fromPythonList.take(3)

[1, 3, 4]

In [8]:
#第三种方式
# 直接创建
sc.range(0,10).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### 注意 collect 前后的类型
`没 collect 的时候是 RDD， collect 之后就是 python 的 list 了`

In [12]:
#rdd 是没有办法打印的  检查类型就是rdd
type(sc.range(0,10))

pyspark.rdd.PipelinedRDD

In [13]:
#加入 collect 就是 list
type(sc.range(0,10).collect())

list

### 几个常用的操作
* map
* flatmap
* filter
* reduce
* reduceByKey
* collect

#### Map

In [10]:
#初始的 RDD
rdd = sc.range(0,10)
rdd.collect() 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
# map function 计算 
#map可以理解成 对每个元素以for循环的方式 进行计算
#使用的时候传入的是  lambda 形式的函数 函数自己定义
rddAfterMap = rdd.map(lambda x : x + 99)
#上面的形式就是 rdd格式
#一旦 collect 就是python 的list
rddAfterMap.collect()

[99, 100, 101, 102, 103, 104, 105, 106, 107, 108]

In [14]:
#当数据非常负责和大的时候，可以重新定义一个函数
#注意函数输入的参数
def myFunction(x):
    return x + 99
a = rdd.map(myFunction)
a.collect()

[99, 100, 101, 102, 103, 104, 105, 106, 107, 108]

In [5]:
 # 初始的 RDD
 # 数组之间的计算
rdd2 = sc.parallelize([(1, 5),(3, 6),(4, 99),(7, 101)])
rdd2.collect()

[(1, 5), (3, 6), (4, 99), (7, 101)]

In [20]:
# 这里进行了计算操作   x[0] + x[1]  x 的第0位加上第一位求和
rdd2.map(lambda x : x[0] + x[1]).collect()

[6, 9, 103, 108]

#### FlatMap (体会 map flap 的区别)
1.Flatmap 要求传入的函数的返回值一定是一个  集合
2.他是返回的结果，打散之后，加入一个集合里面

In [15]:
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
#试验一下  返回值是很多集合，但是打散放在了一起
rdd.flatMap(lambda x : [x,99]).collect()

[0, 99, 1, 99, 2, 99, 3, 99, 4, 99, 5, 99, 6, 99, 7, 99, 8, 99, 9, 99]

In [27]:
#初始化 定义的rdd2
rdd2 = sc.parallelize([(1, 5),(3, 6),(4, 99),(7, 101)])
rdd2.collect()

[(1, 5), (3, 6), (4, 99), (7, 101)]

In [23]:
rdd2.map(lambda x : x).collect() # map不变,没有进行计算，一对一对的

[(1, 5), (3, 6), (4, 99), (7, 101)]

In [32]:
rdd2.flatMap(lambda x : x).collect()  # flatten  把数据打散，变成了单独的

[1, 5, 3, 6, 4, 99, 7, 101]

In [36]:
rdd2.map(lambda x : [x[0] + 1, x[1] + 1]).collect() #map

[[2, 6], [4, 7], [5, 100], [8, 102]]

In [35]:
rdd2.flatMap(lambda x : [x[0] + 1, x[1] + 1]).collect()  # flatten

[2, 6, 4, 7, 5, 100, 8, 102]

In [7]:
# filter ，map 和 flatMap的比较
a= sc.parallelize([1,2,3,3,4])
a.filter(lambda e : e%2 ==0).map(lambda e: e+99).collect()

[101, 103]

In [9]:
a.flatMap(lambda e :[e+99] if e%2 ==0 else []).collect()
#更上面结论一样，

[101, 103]

#### Filter (返回一个 boolean ) 过滤器


In [24]:
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
#只留下奇数
rdd.filter(lambda x:(x%2 != 0)).collect()

[1, 3, 5, 7, 9]

In [44]:
# rdd2  [(1, 5),(3, 6),(4, 99),(7, 101)]
rdd2.filter(lambda x : x[0] & 1 ).collect()  # x[0] 是奇数？

[(1, 5), (3, 6), (7, 101)]

#### Reduce (输入的参数一定是两个)

In [47]:
rdd3 = sc.range(1, 10)
rdd3.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [49]:
rdd3.reduce(lambda x,y : x + y)  # 求和，返回值是 int

45

#### ReduceByKey
操作的元素肯定存在（key,value），至少是个元组

In [29]:
 rdd4 = sc.parallelize([(1, 5),(1, 55), (3, 6), (3, 66), (4, 9), (4, 99), (7, 101)])
# rdd4 = sc.parallelize([[1, 5],[1, 55], [3, 6], [3, 66], [4, 9], [4, 99], [7, 101]]) 也可以
rdd4.collect()

[(1, 5), (1, 55), (3, 6), (3, 66), (4, 9), (4, 99), (7, 101)]

In [55]:
#处理的数据类型（key,value）
#数据里面 key 和key 有一样的，把key一样的聚合在一起，然后对value做一些操作，操作自己定义
# value相加
rdd4.reduceByKey(lambda x, y : x + y).collect()

[(1, 60), (3, 72), (4, 108), (7, 101)]

In [4]:
#combineByKey的用法
rdd2= sc.parallelize([("a", 88),("b",95),("a",91),("b",93),("a",95),("b",98)],2)
rdd2.glom().collect()

[[('a', 88), ('b', 95), ('a', 91)], [('b', 93), ('a', 95), ('b', 98)]]

In [8]:
rdd3 = rdd2.combineByKey(lambda v : (v,1), lambda c,v : (c[0]+ v, v+1), lambda c1, c2 : (c1[0]+c2[0],c1[1]+c2[1]))
rdd3.glom().collect()

[[('b', (286, 100))], [('a', (274, 93))]]

#### Collect 和 Lazy Evaluation (需要去 IDE 演示)

In [58]:
# rdd3 [1, 2, 3, 4, 5, 6, 7, 8, 9]
def f(x):
    print("Mapping Once")
    return x + 1 
rdd3AfterMap = rdd3.map(f) #此时 RDD 并没有真正被操作

In [63]:
rdd3AfterMap.collect() # 此处应该还会输出很多"Mapping Once"， 但是 jupyter 的问题没输出

[2, 3, 4, 5, 6, 7, 8, 9, 10]

#### 还有很多 API 例如 Fold.. GroupByKey ...SortByKey

### DAG

In [65]:
rdd3AfterMap.toDebugString()

b'(1) PythonRDD[48] at collect at <ipython-input-59-e6bd2b5a4286>:1 []\n |  ParallelCollectionRDD[30] at parallelize at PythonRDD.scala:195 []'

In [ ]:
InvertedIndex 倒排索引代码实操 